<a href="https://colab.research.google.com/github/tomyrso/mat281_portfolio/blob/master/docs/homeworks/hw_02_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/fralfaro/MAT281_2024/blob/main/docs/homeworks/hw_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# MAT281 - Tarea N°02


## Instrucciones

1.- Completa tus datos personales (nombre y rol USM) en siguiente celda.


* __Nombre__: Tomas Rojas

* __Rol__: 202110533-6

2.- Debes _subir_ este archivo con tus cambios a tu repositorio personal del curso, incluyendo datos, imágenes, scripts, etc.

3.- Se evaluará:
   - Soluciones
   - Código
   - Al presionar  `Kernel -> Restart Kernel and Run All Cells` deben ejecutarse todas las celdas sin error.
   
4.- Esta Tarea debe ser entregada en **Dos Jupyter Notebooks Distinto**.
   * **Ejemplo**: `hw_02_part_01.ipynb`, `hw_02_part_02.ipynb`.

## II.- Titanic - Machine Learning from Disaster

<img src="https://i.pinimg.com/originals/8c/ef/e7/8cefe799c4d5d2ad4ad7f6524d3838f4.png" width = "400" align="center"/>


El desafío **Titanic - Machine Learning from Disaster** en [Kaggle](https://www.kaggle.com/competitions/titanic/overview/description) invita a predecir qué pasajeros sobrevivieron al naufragio del Titanic mediante un modelo de machine learning. Utiliza datos reales de los pasajeros, como su nombre, edad, género y clase socioeconómica, para explorar patrones de supervivencia y construir un modelo predictivo. Este es uno de los desafíos más populares de Kaggle y un excelente punto de partida para aprender sobre machine learning y análisis de datos.

### Pasos para participar:

1. **Unirse a la competencia**:
   - [Crea una cuenta o inicia sesión en Kaggle](https://www.kaggle.com/account/login?returnUrl=%2Fc%2Ftitanic) y acepta las reglas para acceder a los datos de la competencia.
   
2. **Descargar y explorar los datos**:
   - Descarga los archivos `train.csv` y `test.csv` desde la [página de datos](https://www.kaggle.com/competitions/titanic/data).
   - `train.csv` contiene información de 891 pasajeros, incluyendo si sobrevivieron o no (columna `Survived`). En `test.csv`, se oculta esta columna para que tu modelo prediga la supervivencia de 418 pasajeros adicionales.

3. **Desarrollar el modelo**:
   - Usa `train.csv` para explorar y descubrir patrones, luego entrena un modelo de machine learning que pueda predecir la supervivencia en `test.csv`. Un recurso útil para aprender es el [tutorial de Alexis Cook](https://www.kaggle.com/alexisbcook/titanic-tutorial), que explica paso a paso cómo hacer tu primera predicción.
   - Puedes explorar notebooks de otros participantes para inspiración y técnicas avanzadas en la [sección de notebooks](https://www.kaggle.com/c/titanic/notebooks).

4. **Realizar una predicción y enviar tu archivo**:
   - El archivo CSV de predicciones debe tener dos columnas: `PassengerId` y `Survived`. Puedes consultar un ejemplo en el archivo `gender_submission.csv` disponible en la [página de datos](https://www.kaggle.com/competitions/titanic/data).
   - Sube tu archivo en la sección de envíos y revisa tu puntaje de precisión, que mide el porcentaje de pasajeros que tu modelo predijo correctamente.

5. **Revisar el leaderboard y mejorar el modelo**:
   - Ve tu posición en el [leaderboard](https://www.kaggle.com/c/titanic/leaderboard) y mejora tu modelo basándote en ideas de los foros o pruebas adicionales.

### Ayuda y recursos adicionales:

- [Foro de discusión del Titanic](https://www.kaggle.com/c/titanic/discussion): Un espacio donde puedes hacer preguntas y ver consejos de otros participantes.
- [Vídeo sobre la jerga de Kaggle](https://www.youtube.com/watch?v=sEJHyuWKd-s) por Dr. Rachael Tatman, para entender mejor los términos comunes en Kaggle.
- [Notebooks de la competencia](https://www.kaggle.com/c/titanic/notebooks): Revisa notebooks compartidos para ver cómo otros abordan el desafío.

Este desafío es ideal para principiantes en machine learning y permite practicar desde la limpieza de datos hasta el desarrollo y evaluación de modelos.

In [5]:
## base imports ##
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
gender_submission_df = pd.read_csv('/content/drive/MyDrive/content/gender_submission.csv') #este es un ejemplo de submission
test_df = pd.read_csv('/content/drive/MyDrive/content/test.csv') #datos a testear
train_df = pd.read_csv('/content/drive/MyDrive/content/train.csv') #datos para entrenar

In [8]:
train_df['Survived'].value_counts()

,count
Survived,
0,549
1,342


In [9]:
print('train_df:')
print(train_df.isna().sum())
print('\n')
print('test_df:')
print(test_df.isna().sum())

train_df:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


test_df:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [10]:
train_df['Has_Cabin'] = train_df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
test_df['Has_Cabin'] = test_df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)

In [11]:
full_data = [train_df, test_df]
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    dataset['Age'][np.isnan(dataset['Age'])] = age_avg
    dataset['Age'] = dataset['Age'].astype(int)

<ipython-input-11-409d020f6130>:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataset['Age'][np.isnan(dataset['Age'])] = age_avg
<ipython-input-11-409d020f6130>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [12]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train_df['Fare'].mean())

In [13]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [14]:
drop_elements = ['Cabin']
train_df = train_df.drop(drop_elements,axis=1)
test_df = test_df.drop(drop_elements,axis=1)

In [15]:
print('train_df:')
print(train_df.isna().sum())
print('\n')
print('test_df:')
print(test_df.isna().sum())

train_df:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Has_Cabin      0
dtype: int64


test_df:
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
Has_Cabin      0
dtype: int64


In [16]:
train_df['Ticket_type'] = train_df['Ticket'].apply(lambda x: x[0:3])
train_df['Ticket_type'] = train_df['Ticket_type'].astype('category')
train_df['Ticket_type'] = train_df['Ticket_type'].cat.codes

test_df['Ticket_type'] = test_df['Ticket'].apply(lambda x: x[0:3])
test_df['Ticket_type'] = test_df['Ticket_type'].astype('category')
test_df['Ticket_type'] = test_df['Ticket_type'].cat.codes

train_df = train_df.drop('Ticket',axis=1)
test_df = test_df.drop('Ticket',axis=1)

In [17]:
train_df['Words_Count'] = train_df['Name'].apply(lambda x: len(x.split()))
test_df['Words_Count'] = test_df['Name'].apply(lambda x: len(x.split()))

In [18]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # Si el titulo existe, extraer
    if title_search:
        return title_search.group(1)
    return ""
# Agregar la nueva col
train_df['Title'] = train_df['Name'].apply(get_title)
test_df['Title'] = test_df['Name'].apply(get_title)
# Agrupar los titulos raros como "Rare"

train_df['Title'] = train_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
train_df['Title'] = train_df['Title'].replace('Mlle', 'Miss')
train_df['Title'] = train_df['Title'].replace('Ms', 'Miss')
train_df['Title'] = train_df['Title'].replace('Mme', 'Mrs')

test_df['Title'] = test_df['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
test_df['Title'] = test_df['Title'].replace('Mlle', 'Miss')
test_df['Title'] = test_df['Title'].replace('Ms', 'Miss')
test_df['Title'] = test_df['Title'].replace('Mme', 'Mrs')

In [19]:
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

In [20]:
train_df['Sex'] = train_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
test_df['Sex'] = test_df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)


title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
train_df['Title'] = train_df['Title'].map(title_mapping)
train_df['Title'] = train_df['Title'].fillna(0)
test_df['Title'] = test_df['Title'].map(title_mapping)
test_df['Title'] = test_df['Title'].fillna(0)

train_df['Embarked'] = train_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
test_df['Embarked'] = test_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [21]:
drop_elements = [ 'Name', 'SibSp']
train_df = train_df.drop(drop_elements, axis = 1)
test_df  = test_df.drop(drop_elements, axis = 1)

In [22]:
svm = SVC(kernel='rbf',C=1000)
grad = GradientBoostingClassifier(n_estimators=10000,learning_rate=0.0005,max_depth=3)
X_train,X_test,y_train,y_test = train_test_split(train_df.drop(['PassengerId','Survived'],axis=1),train_df['Survived'],test_size=0.1)
svm.fit(X_train,y_train)
print('train score:',svm.score(X_train,y_train))
print('test score:',svm.score(X_test,y_test))
grad.fit(X_train,y_train)
print('train score:',grad.score(X_train,y_train))
print('test score:',grad.score(X_test,y_test))

train score: 0.83645443196005
test score: 0.8777777777777778
train score: 0.8701622971285893
test score: 0.9


In [24]:
columnas = train_df.drop(['PassengerId','Survived'],axis=1).columns.to_list()
x_test = test_df[columnas]
y_pred = svm.predict(x_test)
ids = test_df['PassengerId']
submission = pd.DataFrame({'PassengerId':ids,'Survived':y_pred})
submission.to_csv('submission.csv',index=False)